In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [0]:
df1=pd.read_csv("art_1.csv").set_index('Unnamed: 0')
df2=pd.read_csv("art_2.csv").set_index('Unnamed: 0')

In [0]:
news_data=pd.concat([df1,df2]).drop(columns=['date']).reset_index(drop=True)

In [163]:
news_data

,description,text
0,"Stores offer coronavirus hours for seniors, at...",Get all the latest news on coronavirus and mor...
1,"Why male business leaders use Botox, youth-ind...",Powerful industry leaders are relying on nonin...
2,"US GDP expected to fall 12%, federal deficit t...",The United States' gross domestic product is e...
3,How coronavirus could damage Social Security,Get all the latest news on coronavirus and mor...
4,"Boeing ends deal, angering Brazilian jet maker...",CHICAGO — Boeing announced Saturday it termina...
...,...,...
5124,COVID-19 survivor State Rep. Karen Whitsett de...,Get all the latest news on coronavirus and mor...
5125,'Tiger King': Lawyer for Carole Baskin's missi...,"An attorney for the missing husband of ""Tiger ..."
5126,Former Nevada attorney general says MI Gov. Wh...,Get all the latest news on coronavirus and mor...
5127,Former Rep. Corinne Brown released from prison...,Get all the latest news on coronavirus and mor...


In [0]:
list_articles=[]
for i in range(len(news_data)):
    list_articles.append(str(news_data['description'][i])+"  "+str(news_data['text'][i]))

In [165]:
!pip install vaderSentiment #library for sentiment analysis

In [0]:
import operator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [167]:
emote_list=[]
vs=[]
for sentence in list_articles:
    vs.append(analyzer.polarity_scores(sentence))
print('loading...')
vs=[{k: v for k, v in d.items() if k != 'compound'} for d in vs]

loading...


In [0]:
for item in vs:
    if item['neg']<0.2:
        if item['pos']<0.2:
            emote_list.append('neu')
        else:
             emote_list.append('pos')
    else:
        if item['pos']>0.2:
            emote_list.append('neu')
        else:
            emote_list.append('neg')

In [0]:
news_data['emotion']=emote_list

In [170]:
news_data[news_data['emotion']=='pos']

,description,text,emotion
41,Bud Light helps raise coronavirus relief effor...,NaN,pos
209,Bud Light helps raise coronavirus relief effor...,NaN,pos
349,Is there a process for licensing content?,"To license footage, please reach out to Archiv...",pos
355,Sen. Kennedy: Early Virus Models Made Late Nig...,Louisiana Senator John Kennedy called in to th...,pos
424,Bud Light helps raise coronavirus relief effor...,NaN,pos
...,...,...,...
5104,Georgia state senator's 'second chance at life...,Get all the latest news on coronavirus and mor...,pos
5112,Global movement to pray for health care worker...,Get all the latest news on coronavirus and mor...,pos
5114,Georgia state senator's 'second chance at life...,Get all the latest news on coronavirus and mor...,pos
5115,"Justice with Judge Jeanine - Saturday, April 25",NaN,pos


In [0]:
from collections import Counter
from imblearn.over_sampling import SMOTE

X_train=news_data.iloc[:,:2]
y_train=news_data['emotion']

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
count_vect = CountVectorizer()
le = preprocessing.LabelEncoder()
X_train_counts = count_vect.fit_transform(list_articles)
le.fit(y_train)
y_train=le.transform(y_train)
labels=le.classes_

In [173]:
y_train

array([1, 1, 1, ..., 1, 1, 1])

In [0]:
from sklearn.neural_network import MLPClassifier
df3=pd.read_csv("art_3.csv").set_index('Unnamed: 0').drop(columns=['date'])
articles_df3=[]
for i in range(len(df3)):
    articles_df3.append(str(df3['description'][i])+"  "+str(df3['text'][i]))

In [0]:
X_test_counts = count_vect.transform(articles_df3)
model = MLPClassifier().fit(X_train_counts, y_train)
y_pred = model.predict(X_test_counts)

In [145]:
X_train_counts

<5129x12850 sparse matrix of type '<class 'numpy.int64'>'
	with 1197017 stored elements in Compressed Sparse Row format>

In [146]:
X_test_counts

<1520x12850 sparse matrix of type '<class 'numpy.int64'>'
	with 366931 stored elements in Compressed Sparse Row format>

In [148]:
df3['pred']=y_pred
df3

,description,text,pred
Unnamed: 0,,,
0,Outnumbered Overtime,"Capitalizing on the strength of Outnumbered, H...",1
1,Sunday Morning Futures,Follow Maria Bartiromo as she brings big busin...,1
2,The Next Revolution,The Next Revolution with Steve Hilton focuses ...,1
3,Outnumbered Overtime,"Capitalizing on the strength of Outnumbered, H...",1
4,Sunday Morning Futures,Follow Maria Bartiromo as she brings big busin...,1
...,...,...,...
1515,COVID-19 survivor State Rep. Karen Whitsett de...,Get all the latest news on coronavirus and mor...,1
1516,'Tiger King': Lawyer for Carole Baskin's missi...,"An attorney for the missing husband of ""Tiger ...",1
1517,Former Nevada attorney general says MI Gov. Wh...,Get all the latest news on coronavirus and mor...,1


## viral content

In [176]:
df3[df3['pred']==2]

,description,text,pred
Unnamed: 0,,,
19,Is there a process for licensing content?,"To license footage, please reach out to Archiv...",2
25,Sen. Kennedy: Early Virus Models Made Late Nig...,Louisiana Senator John Kennedy called in to th...,2
93,Bud Light helps raise coronavirus relief effor...,NaN,2
261,Bud Light helps raise coronavirus relief effor...,NaN,2
401,Is there a process for licensing content?,"To license footage, please reach out to Archiv...",2
407,Sen. Kennedy: Early Virus Models Made Late Nig...,Louisiana Senator John Kennedy called in to th...,2
444,Is there a process for licensing content?,"To license footage, please reach out to Archiv...",2
488,Bud Light helps raise coronavirus relief effor...,NaN,2
647,A Tiger King Investigation with Nancy Grace: S...,NaN,2


In [158]:
list(le.classes_)

[0, 1, 2]

In [0]:
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

In [175]:
le_name_mapping

{'neg': 0, 'neu': 1, 'pos': 2}